In [ ]:
import numpy as np
import matplotlib.pyplot as plt  
%matplotlib inline
# from IPython.display import display, HTML

from pykat import finesse
from pykat.commands import *
pykat.init_pykat_plotting(dpi=90)

from pprint import pprint as pprint

In [ ]:
# base of configurations, without Southern section
basekat = finesse.kat()
basecode = """
const Pin 125 # laser power
# lengths in metres
const lx 1.6
const ly 1.55
const Lx 4k
const Ly 4k
const lprc 53
# mirrors
const Tprm 0.3
const Rprm 0.7
const Titm 0.014
const Ritm 0.986
const Tetm 6E-6
const Retm 0.999994

# laser
l laser $Pin 0 nLaser

# power recycling
m PRM $Rprm $Tprm 0 nLaser nPRM
s sBStoPRM $lprc nPRM nBSi

# central beam splitter
bs BS 0.5 0.5 0 0 nBSi nBSr nBSt nBSo
s sBStoYarm $ly nBSr nY0
s sBStoXarm $lx nBSt nX0

# Y arm (perpendicular)
m mY1 $Ritm $Titm 0 nY0 nY1
s sY $Ly nY1 nY2
m mY2 $Retm $Tetm 0 nY2 nY3

# X arm (parallel)
m mX1 $Ritm $Titm 0 nX0 nX1
s sX $Lx nX1 nX2
m mX2 $Retm $Tetm 0 nX2 nX3
"""
basekat.parse(basecode)

# pprint((basekat.components, basekat.detectors, basekat.commands))

In [ ]:
def create_kat1(lsrc):
    """ configuration 1: adding in SRM section
    lsrc in m
    """
    kat1 = deepcopy(basekat)
    kat1code = """
    # signal recycling
    const lsrc {0}
    const Tsrm 0.2
    const Rsrm 0.8
    
    s sBStoSRM $lsrc nBSo nSR
    m SRM $Rsrm $Tsrm 0 nSR nDark
    """.format(lsrc)
    kat1.parse(kat1code)
    
    return kat1

In [ ]:
def create_kat2(lsrc, gain=10):
    """configuration 2: adding in SRM section with nle halfway
    lsrc in m, gain in field dB
    """
    kat2 = deepcopy(basekat)
    kat2code = """
    # signal recycling
    const lsrc {0}
    const halflsrc {1}
    const Tsrm 0.2
    const Rsrm 0.8
    
    s sBStonle $halflsrc nBSo nnle11
    nle nle1 {2} 0 nnle11 nnle12 # gain defaults to 10 dB, angle = 0
    s snletoSRM $halflsrc nnle12 nSR    
    m SRM $Rsrm $Tsrm 0 nSR nDark
    """.format(lsrc, lsrc/2, gain)
    kat2.parse(kat2code)
    
    return kat2

In [ ]:
def noise_transfer(kat):
    """calculates QM noise transfer function for created kat1 or kat2"""
    noisekat = deepcopy(kat)
    noisekat.verbose = False
    noisecode = """
    pd darkport nDark
    
    fsig noise 100 # enters through every open port and lossy optic
    xaxis noise f lin 1 10G 1000
    """
    noisekat.parse(noisecode)
    out = noisekat.run()
    
    return out

In [ ]:
def gw_transfer(kat):
    """calculates GW signal transfer function for created kat1 or kat2
    currently shakes mX2 at varying frequencies
    
    # how to do differential tuning with fsig?
    
    """
    signalkat = deepcopy(kat)
    signalkat.verbose = False
    signalcode = """
    pd1 darkport 10k 0 nDark
            
    fsig sig1 mX2 10k 0
    xaxis sig1 f lin 1 10G 1000
    
    put darkport f1 $x1
    """
    signalkat.parse(signalcode)
    out = signalkat.run()
    
    return out    

In [ ]:
# perhaps should define a plotter class?

kat1c1 = create_kat1(53)
kat1c2 = create_kat1(2000)
kat2c1 = create_kat2(53)
kat2c2 = create_kat2(2000)

title1c1 = "configuration 1, case 1"
title1c2 = "configuration 1, case 2"
title2c1 = "configuration 2, case 1"
title2c2 = "configuration 2, case 2"

In [ ]:
noise_transfer(kat1c1).plot(title=title1c1)
noise_transfer(kat1c2).plot(title=title1c2)
noise_transfer(kat2c1).plot(title=title2c1)
noise_transfer(kat2c2).plot(title=title2c2)

In [ ]:
gw_transfer(kat1c1).plot(title=title1c1)
gw_transfer(kat1c2).plot(title=title1c2)
gw_transfer(kat2c1).plot(title=title2c1)
gw_transfer(kat2c2).plot(title=title2c2)